In [4]:
from dataretrieval import nwis
from IPython.display import display
from matplotlib import ticker
import csv
import os
import re

In [51]:
def extract_unique_15_digit_numbers(file_path):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Find all 15-digit numbers using regular expression
    numbers = re.findall(r'\b\d{15}\b', content)
    
    # Convert to a set to get unique numbers, then back to a list
    unique_numbers = list(set(numbers))
    
    return unique_numbers

# Example usage:
file_path = 'data/2020TempretureData.rdb'
unique_15_digit_numbers = list(map(int,extract_unique_15_digit_numbers(file_path)))
print(" Found this many sites with data" ,len(unique_15_digit_numbers))

 Found this many sites with data 448


In [52]:
# USGS	435723092171801	2022-01-18 23:15	CST	10.4	A

def convert_to_float_if_possible(d):
    for key, value in d.items():
        try:
            # Try to convert the value to a float
            d[key] = float(value)
        except:
            # If the conversion fails, leave the value as is
            pass
    return d


def convert_to_float(value):
    try:
        # Try to convert the value to a float
        return float(value)
    except:
        # If the conversion fails, leave the value as is
        return value



def loadSiteData(data,fn):

    cecInData = os.path.join("data",fn )    

    with open(cecInData, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if len(row["SiteNumber"]) > 12:
                site_num = int(row["SiteNumber"])
                row =  convert_to_float_if_possible(row)
                row["SiteNumber"] = site_num
                data[site_num] = row
                data[site_num]["temps"] = []

    return data

#cec_module_db2 = pvlib.pvsystem.retrieve_sam(path='data/CECModules2023.csv')
site_data = {}
#SiteNumber, SiteName, SiteCategory, SiteAgency, SiteLongitude, SiteLatitude, 
site_data = loadSiteData(site_data,"NWISMapperExport.csv")

In [55]:

def extract_tempData(file_path,site_data):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Find all 15-digit numbers using regular expression
    temps_regex=r"(\d{15})\b.*\b(\d\d?.\d)\tA$"

    for mo in re.finditer(temps_regex, content, re.MULTILINE):
        site = int(mo.group(1))
        temp = convert_to_float(mo.group(2))
        if site in site_data:
            # print("found ",site)
            site_data[site]['temps'].append(temp)
    # Convert to a set to get unique numbers, then back to a list
    
    return site_data

# Example usage:
file_path = 'data/2020TempretureData.rdb'
site_data = extract_tempData(file_path,site_data)
print(" Found this many sites with data" ,len(site_data))

print(site_data[unique_15_digit_numbers[0]])

 Found this many sites with data 2050
{'SiteNumber': 324007107095501, 'SiteName': '19S.03W.08.423 HD-1', 'SiteCategory': 'GW', 'SiteAgency': 'USGS', 'SiteLongitude': -107.1653333, 'SiteLatitude': 32.66863889, 'SiteNWISURL': 'https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=324007107095501', 'temps': []}


In [56]:
import statistics 
import json
gj = {}

for site_num in unique_15_digit_numbers: 
    if site_num in site_data and len(site_data[site_num]['temps'])>0:
        gj[site_num] ={
        "type": "Feature",
        "properties": {
            "name": site_data[site_num]['SiteName'],
            "id": site_num,
            "temperature": statistics.mean(site_data[site_num]['temps'])
        },
        "geometry": {
            "type": "Point",
            "coordinates": [site_data[site_num]['SiteLongitude'],site_data[site_num]['SiteLatitude']]
        }
}
with open('data/groundTemps.json', 'w', newline='') as jsonFile:    
    json.dump(gj,jsonFile,indent='\t')

In [1]:
# "00010," # Water Temp In C
# "00011," # Water Temp In F
#Parameter Code 72019 = Depth to water level, ft below land surface
#Parameter Code 99268	Depth of sensor below water surface, not corrected for barometric pressure variations, ft
# Parameter Code 72147  	Depth of sensor below water surface, ft
# statReportType="daily",

x3 = nwis.get_stats(sites=unique_15_digit_numbers[10:20], parameterCd=["00010", "72147","72019" ],
                    statReportType="daily", statTypeCd=["mean"],
                    startDt="2022-01-01", endDt="2022-01-02")
display(x3[0])

NameError: name 'nwis' is not defined